<a href="https://colab.research.google.com/github/cdenq/aws-cloud-etl-shopper-data-anaylsis/blob/main/ETL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Downloading spark versions
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("chrisETL1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://chrisdenq-aws-etl-data-bucket.s3.amazonaws.com/amazon_reviews_us_Shoes_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv"), inferSchema=True, sep = '\t')
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [ ]:
# import rest of libraries
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType

In [ ]:
# connect to PostGres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-13 12:13:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.23MB/s    in 0.7s    

2022-03-13 12:13:53 (1.23 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



# Preprocessing + ETL

In [ ]:
# count of untouched, then of dropped
df.count()
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

149086
149080
149080


In [ ]:
# check schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#isolate relevant table and convert to date
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).withColumn("review_date",col("review_date").cast(DateType()))
review_id_table.show(5)
review_id_table.printSchema()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RR8P8H4144RUD|   28726648|B007V6EVY2|     910961751| 2015-08-31|
|R1QGEWKB99QW3G|   16943853|B00EPLSMGY|     725850175| 2015-08-31|
|R1E6DFR0YQVPCB|   13097243|B00BWDH3VS|     473048287| 2015-08-28|
| RFDF7ZIKH9PVL|   46281218|B00BWDH368|     473048287| 2015-08-25|
| RWGW40MZS35WQ|   29640138|B0066AZGJI|     480731329| 2015-08-22|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
#drop dupes
products = df.select(["product_id", "product_title"]).dropDuplicates(['product_id', 'product_title'])
products.show(5)
products.printSchema()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CHQKNUM|Amazon.com Gift C...|
|B00CRQ4ZOM|Amazon Gift Card ...|
|B00H5BMZ32|Amazon eGift Card...|
|BT00CTP84S|Amazon.com Gift C...|
|B004LLILN2|Amazon eGift Card...|
+----------+--------------------+
only showing top 5 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# isolate customers and count
customers = df.select(["customer_id"])
customers = customers.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").dropDuplicates(['customer_id'])
customers.show(5)
customers.printSchema()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35594973|             1|
|   18961565|             1|
|   45120109|             1|
|   22720037|             1|
|   45528330|             1|
+-----------+--------------+
only showing top 5 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
mode="append"
jdbc_url = "jdbc:postgresql://chrisdenq-rds.clufauwyhohi.us-east-1.rds.amazonaws.com:5432/chrisdenq-rds"
config = {"user":"root",
          "password": "=",
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)